In [22]:
import pandas as pd

**Reading in the Million Song Dataset downloaded Kaggle

In [46]:
df = pd.read_csv("E:/DS Project/Music Recommendation System/archive/spotify_millsongdata.csv")

In [47]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [48]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [49]:
df.shape

(57650, 4)

In [50]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [51]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [52]:
df.head(10)

,artist,song,text
0,Celine Dion,It's All Coming Back To Me Now,There were nights when the wind was so cold \...
1,Flo-Rida,Come With Me,I met a lot of women on the road \r\nI'm pipi...
2,INXS,Kill The Pain,Put the red lights on \r\nDon't tell them tha...
3,Prince,Calhoun Square,I don't like this much to drink \r\nI was the...
4,Rascal Flatts,Come Wake Me Up,I can usually drink you right off of my mind ...
5,Snoop Dogg,Betta Days,"Celebrate, grab a drink and put a blunt in the..."
6,Josh Groban,Au Jardin Des Sans Pourquoi,Retournons avant la guerre \r\nAvant la fin ...
7,Katy Perry,Black And Gold,If the fish swam out of the ocean \r\nAnd gre...
8,Cheap Trick,Never Had A Lot To Lose,"I don't need no money, honey \r\nFortune and ..."
9,Beach Boys,A Thing Or Two,When I see my baby \r\nWhen I see my little g...


In [53]:
df['text'][0]

"There were nights when the wind was so cold  \r\nThat my body froze in bed  \r\nIf I just listened to it  \r\nRight outside the window  \r\n  \r\nThere were days when the sun was so cruel  \r\nThat all the tears turned to dust  \r\nAnd I just knew my eyes were  \r\nDrying up forever  \r\n  \r\nI finished crying in the instant that you left  \r\nAnd I can't remember where or when or how  \r\nAnd I banished every memory you and I had ever made  \r\n  \r\nBut when you touch me like this  \r\nAnd you hold me like that  \r\nI just have to admit  \r\nThat it's all coming back to me  \r\nWhen I touch you like this  \r\nAnd I hold you like that  \r\nIt's so hard to believe but  \r\nIt's all coming back to me  \r\n(It's all coming back, it's all coming back to me now)  \r\n  \r\nThere were moments of gold  \r\nAnd there were flashes of light  \r\nThere were things I'd never do again  \r\nBut then they'd always seemed right  \r\nThere were nights of endless pleasure  \r\nIt was more than any la

In [54]:
df.shape

(5000, 3)

In [56]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [57]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [60]:
tokenization('you are beautiful')

'you are beauti'

In [58]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [63]:
similarity[0]

array([1.        , 0.10795466, 0.0338362 , ..., 0.08102038, 0.03140236,
       0.02224938])

In [66]:
df[df['song'] == 'Come With Me']

,artist,song,text
1,Flo-Rida,Come With Me,i met a lot of women on the road i 'm pipe thi...


In [67]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [68]:
recommendation('Come With Me')

["Don't Wanna Be In Love",
 'More And More',
 'All The Small Things (Blink 182 Cover)',
 'The Man With The Horn',
 'I Wanna Be',
 'I Wanna Believe You',
 "I've Just Seen A Face",
 'Birthday Song',
 'One Life',
 'I Wanna Get Back With You',
 'Put That Thang On U',
 'Use Your Love',
 'Make It Wit Chu',
 'Just Wanna Be With You',
 'High For This',
 "Don't Wanna Dance Alone",
 'Fight',
 'Feels Good At First',
 'Novocaine',
 'Drive']

In [69]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))